In [0]:
##fact_payment###

print("\n Create fact_payment...")

fact_payment = spark.sql("""
    SELECT 
        -- Primary Key
        p.payment_id,
        
        -- Foreign Keys to Dimensions  
        p.rider_id as rider_key,
        DATE_FORMAT(p.payment_date, 'yyyyMMdd') as date_key,
        
        -- FACTS - Core Business Metrics
        p.amount as payment_amount,
        
        -- Time Attributes for Quick Analysis
        YEAR(p.payment_date) as payment_year,
        MONTH(p.payment_date) as payment_month,
        QUARTER(p.payment_date) as payment_quarter,
        
        -- Derived Business Fields
        CASE 
            WHEN r.is_member = true THEN 'Member Payment'
            ELSE 'Casual Payment'
        END as payment_type,
        
        -- Member age at time of payment for demographic analysis
        FLOOR(DATEDIFF(p.payment_date, r.birthday) / 365.25) as member_age_at_payment,
        
        -- Account tenure at payment time
        ROUND(DATEDIFF(p.payment_date, r.account_start_date) / 365.25, 2) as account_tenure_at_payment
        
    FROM bronze_payments p
    INNER JOIN bronze_riders r ON p.rider_id = r.rider_id
    WHERE p.payment_date IS NOT NULL
      AND p.amount > 0  -- Data quality check
""")

# Write with partitioning by year for performance
fact_payment.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .partitionBy("payment_year") \
    .saveAsTable("gold_fact_payment")

payment_count = fact_payment.count()
print(f" fact_payment created: {payment_count:,} rows")

# Show sample with business metrics
print("\n Sample Payment Facts with Business metric:")
fact_payment.select("payment_id", "rider_key", "payment_amount", "payment_type", 
                    "member_age_at_payment", "account_tenure_at_payment").show(5)

print("\n FACT TABLES COMPLETED!")
print("="*70)


 Create fact_payment...
 fact_payment created: 1,946,573 rows

 Sample Payment Facts with Business metric:
+----------+---------+--------------+--------------+---------------------+-------------------------+
|payment_id|rider_key|payment_amount|  payment_type|member_age_at_payment|account_tenure_at_payment|
+----------+---------+--------------+--------------+---------------------+-------------------------+
|        35|     1001|           9.0|Member Payment|                   43|                     0.08|
|        36|     1001|           9.0|Member Payment|                   43|                     0.17|
|        37|     1001|           9.0|Member Payment|                   43|                     0.25|
|        38|     1001|           9.0|Member Payment|                   43|                     0.33|
|        39|     1001|           9.0|Member Payment|                   43|                     0.42|
+----------+---------+--------------+--------------+---------------------+----------